In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Name - Rev.get_R_est
#__author__ = 'Prakhar MISRA'
# Created 8/18/2019
# Last edit 8/18/2010

# code to answer reviwer commment regarding the inability of the model toaccount for secondary PM2.5 
# hence following Upadhaye and dey 2018, we will also include meteorology 

# Function to multiply the cities with corresponding beta to find the Rest
# also known as get beta


# Important resource

# following course by PSU teaches statistics with derived examples
# Confidence interval -  https://newonlinecourses.science.psu.edu/stat414/node/297/
# prediction interval - https://newonlinecourses.science.psu.edu/stat414/node/298/
# CI for Multi linear regression https://newonlinecourses.science.psu.edu/stat501/lesson/7/7.1

# for bayesian 
# More advanced , posterioir predictive distribution for Bayesian 
# https://baezortega.github.io/2018/08/06/robust_regression/

import pandas as pd
import os
import scipy as sp
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
from datetime import timedelta, date
import matplotlib.dates as mdates
import time
import netCDF4
import datetime as dt
from netCDF4 import MFDataset
from netCDF4 import num2date
import datetime as dt  # Python standard library datetime  module
from glob import glob

currdir = os.getcwd()
import seaborn as sns
sns.set_style("whitegrid")



%matplotlib notebook

gpath = r"D:\Research\Codes_W\GoogleTrends\\"
currdir = os.getcwd()



In [9]:
# get path of meteor
meteorpath = os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", "s_r_fromREASmodel", 'allAY20180212IDW2setclean_meteor_sr.csv')

In [10]:
def get_betaRH(RH):
    # beta refers to mass extinction efficiency to correct AOD by RH
    # based on Tropospheric AOT from the GOCART model and comaparisons with satelluite and sun photometer measurements (chin, ginoux, kinne 2002)
    #returning beta values for hydrophiolic OC only
    # y = 0.0004x2 - 0.0207x + 1.0606

    RH = int(RH)
    if RH < 20:
        beta = 1

    if (RH >=20) & (RH <40):
        beta = 1.25

    if (RH >=40) & (RH <60):
        beta = 1.33

    if (RH >=60) & (RH <70):
        beta = 1.6
    
    if (RH >=70) & (RH <80):
        beta = 1.7
        

    if RH>=80:
        beta = 0.0004*RH**2 - 0.0207*RH + 1.0606

    return beta

In [20]:
# open the metoer path and correct the R by sulfate function
df_R = pd.read_csv(meteorpath, header = 0)

# first undo the previous beta correction
df_R["R_undo"] =  df_R["R"]*df_R["beta"]


df_R['beta_sulfate'] = df_R['rhum']

#calling coefficient factor
df_R['beta_sulfate'] = df_R['beta_sulfate'].apply(lambda x: get_betaRH(x))

# simple correction
df_R['R'] = df_R['R_undo']/(df_R['beta_sulfate'])




In [21]:
# save the new R corrected by sulfate aerosol
df_R.to_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", "s_r_fromREASmodel", "sulphate_aerosol_correction", 'allAY20180212IDW2setclean_meteor_sr.csv'))

In [25]:
df_20012011 = df_R[(df_R.year == 2001) | (df_R.year == 2011)]
df_20012011.to_csv(os.path.join(currdir, "my_bayesian",  "HBM_city_tier_20180116","report20180212_IDW2_20190814", "meteorology_model", "s_r_fromREASmodel", "sulphate_aerosol_correction", 'allAY20180212IDW2setclean20012011.csv'))